In [ ]:
import os
import zipfile
from urllib.request import urlretrieve
from src.DataScienceProject import logger

In [18]:
%pwd

'/Users/drdre/Desktop/ML/mlops_proj'

In [ ]:
#os.chdir("..")

In [14]:
%ls

1_data_ingestion.ipynb  research.ipynb


In [20]:
from src.DataScienceProject import logger

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:str
    unzip_dir: Path



In [22]:
from src.DataScienceProject import * 
from src.DataScienceProject.utils.common import read_yaml,create_directories
from src.DataScienceProject.constants import (
    CONFIG_FILE_PATH,
    PARAMS_FILE_PATH,
    SCHEMA_FILE_PATH,
)
class ConfigurationManager:
    def __init__(self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config




In [23]:
##Data ingestion component

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    #downlaoding the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [24]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-08 04:50:23,695: INFO: common: yaml file: config/config.yaml loaded success]
[2026-01-08 04:50:23,698: INFO: common: yaml file: params.yaml loaded success]
[2026-01-08 04:50:23,699: INFO: common: yaml file: schema.yaml loaded success]
[2026-01-08 04:50:23,700: INFO: common: create directory at:artifacts]
[2026-01-08 04:50:23,700: INFO: common: create directory at:artifacts/data_ingestion]
[2026-01-08 04:50:23,803: INFO: 2879312233: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A13E:EC6B2:6F83:8669:695F7DDF
Accept-Ranges: bytes
Date: Thu, 08 Jan 2026 09:50:23 GMT
V